In [64]:
import pandas as pd
import numpy as np
import lasio

las = lasio.read(r'../../data_well_log/las_files_Lithostrat_data/7_1-1.las')
df = las.df()
df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 17982 entries, 82.600799561 to 2815.7127996
Data columns (total 20 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   FORCE_2020_LITHOFACIES_CONFIDENCE  16241 non-null  float64
 1   FORCE_2020_LITHOFACIES_LITHOLOGY   16242 non-null  float64
 2   CALI                               16270 non-null  float64
 3   BS                                 17947 non-null  float64
 4   DCAL                               16272 non-null  float64
 5   MUDWEIGHT                          15724 non-null  float64
 6   ROP                                17343 non-null  float64
 7   RDEP                               16121 non-null  float64
 8   RSHA                               16115 non-null  float64
 9   RMED                               16152 non-null  float64
 10  SP                                 16151 non-null  float64
 11  DTC                               

In [47]:
las = lasio.read(r'../../data_well_log/las_files_Lithostrat_data/35_4-1.las')
df = las.df()
df.columns

Index(['FORCE_2020_LITHOFACIES_CONFIDENCE', 'FORCE_2020_LITHOFACIES_LITHOLOGY',
       'CALI', 'BS', 'ROP', 'RDEP', 'RMED', 'DTS', 'DTC', 'NPHI', 'PEF', 'GR',
       'RHOB', 'DRHO', 'DEPTH_MD', 'X_LOC', 'Y_LOC', 'Z_LOC'],
      dtype='object')

In [55]:
log_curves = ['CALI', 'BS', 'DCAL', 'ROP', 'RDEP', 'RSHA', 'RMED', 'SP', 'DTS', 'DTC', 'NPHI', 'GR', 'RHOB', 'DRHO']

for curve in log_curves:
    if curve not in df.columns:
        df[curve] = 0


X = df.fillna(0)
X = X[log_curves]
X = X.apply(lambda x: (x - x.min()) / (x.max() - x.min()))
X = X.fillna(0)


In [56]:
X

,CALI,BS,DCAL,ROP,RDEP,RSHA,RMED,SP,DTS,DTC,NPHI,GR,RHOB,DRHO
DEPT,,,,,,,,,,,,,,
403.604,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.685957
403.756,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.897596,0.000000,0.0,0.685957
403.908,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.857792,0.000000,0.0,0.685957
404.060,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.791115,0.000000,0.0,0.685957
404.212,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.817098,0.000000,0.0,0.685957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4957.828,0.0,0.236111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.382298,0.0,0.685957
4957.980,0.0,0.236111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.384776,0.0,0.685957
4958.132,0.0,0.236111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.377819,0.0,0.685957


D: The number of depth points.

N: The number of logging curves (features).
data: The values of the selected logging curves (logging_curves) from the DataFrame (X) converted into a NumPy array.

input_tensor: Reshaped tensor with dimensions (D, 1, N)

values along the x-axis correspond to the different logging curves, and the values along the y-axis correspond to the depth points.

In [57]:
# Reshape the DataFrame. The input data: 1D 
D = len(X.index.values)
N = len(X.columns)

print(f'D: {D}')
print(f'N: {N}')
data = X.to_numpy()

print(data)

# print(depth_values)
input_tensor = np.reshape(data, (D, 1, N))
print(input_tensor.shape)


np.save('1D-image-SegLog/test_well.npy', input_tensor)


D: 29967
N: 14
[[0.         0.         0.         ... 0.         0.         0.68595694]
 [0.         0.         0.         ... 0.         0.         0.68595694]
 [0.         0.         0.         ... 0.         0.         0.68595694]
 ...
 [0.         0.23611111 0.         ... 0.37781945 0.         0.68595694]
 [0.         0.23611111 0.         ... 0.37157599 0.         0.68595694]
 [0.         0.23611111 0.         ... 0.37777271 0.         0.68595694]]
(29967, 1, 14)


In [60]:
# Normalize the input tensor
mean = np.mean(data)
std = np.std(data)
input_tensor = (data - mean) / std

# Reshape the input tensor
input_tensor = np.reshape(input_tensor, (D, N))

# Convert the input tensor to PIL image
image = Image.fromarray(input_tensor.astype(np.uint8), mode='L')
image.save('1d_image_2.png')

In [69]:
lithology_numbers = {30000: 0,
                 65030: 1,
                 65000: 2,
                 80000: 3,
                 74000: 4,
                 70000: 5,
                 70032: 6,
                 88000: 7,
                 86000: 8,
                 99000: 9,
                 90000: 10,
                 93000: 11,
                 12345: 12}

labels = df['FORCE_2020_LITHOFACIES_LITHOLOGY'].fillna(12345).astype(int)
# Map numeric labels to lithology names
y = np.array([lithology_numbers[label] for label in labels])


# Get the total number of depth points (D) and number of lithology classes (C)
D = len(y)
C = len(lithology_numbers)

# Create an empty lithology label image
label_image = np.zeros((D, C), dtype=np.float32)

# Set the corresponding class index to 1 for each depth point
for i in range(D):
    label_image[i, y[i]] = 1

# Convert the label image to PIL image
label_image_pil = Image.fromarray((label_image * 255).astype(np.uint8), mode='L')
label_image_pil.save('label_img.png')